In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:15 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://jlozano1990-challenge-bucket.s3.amazonaws.com/amazon_reviews_us_Watches_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Watches_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [4]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3O9SGZBVQBV76|          5|            0|          0|   N|                Y|
| RKH8BNC3L5DLF|          5|            0|          0|   N|                Y|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|                Y|
|R31U3UH5AZ42LL|          5|            0|          0|   N|                Y|
|R2SV659OUJ945Y|          4|            0|          0|   N|                Y|
| RA51CP8TR5A2L|          5|            0|          0|   N|                Y|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|                Y|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|                N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|                Y|
| RX27XIIWY5JPB|          4|            0|          0|   N|     

In [5]:
# Filter the DataFrame to only show when have over 20 total votes
helpful_vine_df = vine_df.filter("total_votes>20")
helpful_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
| R6F9VY91ADPLA|          1|            8|         30|   N|                N|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|                Y|
|R2RB1HML8N712P|          5|           26|         28|   N|                Y|
| ROXA8XP5EI2KK|          5|           48|         49|   N|                N|
|R1B7M0OP3UNP6O|          5|           49|         52|   Y|     

In [6]:
# Convert columns to integers
from pyspark.sql.types import IntegerType
helpful_vine_df = helpful_vine_df.withColumn("helpful_votes", helpful_vine_df["helpful_votes"].cast(IntegerType()))
helpful_vine_df = helpful_vine_df.withColumn("total_votes", helpful_vine_df["total_votes"].cast(IntegerType()))

In [7]:
# Filter dataframe to only show where the number of helpful_votes divided by total_votes is equal to or greater than 50%
more_helpful_vine_df = helpful_vine_df.filter("helpful_votes/total_votes >= 0.5")
more_helpful_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|                Y|
|R2RB1HML8N712P|          5|           26|         28|   N|                Y|
| ROXA8XP5EI2KK|          5|           48|         49|   N|                N|
|R1B7M0OP3UNP6O|          5|           49|         52|   Y|                N|
|R2N92CUM7WBA1A|          5|           27|         28|   N|     

In [8]:
# Show when review was paid
vine_review_Y_df = more_helpful_vine_df.filter(more_helpful_vine_df["vine"] == "Y")
vine_review_Y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B7M0OP3UNP6O|          5|           49|         52|   Y|                N|
|R2UUV4UGGYMQG8|          5|           34|         39|   Y|                N|
| R9K0LZV2BK9YY|          4|           37|         39|   Y|                N|
|R2OVFLNEUEGTJM|          3|           18|         25|   Y|                N|
| RBE09ELJ77LQ0|          5|           44|         45|   Y|                N|
|R3867T8AIJJHM6|          5|           26|         27|   Y|                N|
|R1FNVUXPU63WOZ|          4|           43|         48|   Y|                N|
|R25XGG2G12SE1Z|          4|           20|         23|   Y|                N|
| R3JKU4HRDFZDH|          4|           27|         30|   Y|                N|
|R2PQYOCJXRB1BF|          5|           26|         28|   Y|     

In [9]:
# Show when review was NOT paid
vine_review_N_df = more_helpful_vine_df.filter(more_helpful_vine_df["vine"] == "N")
vine_review_N_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|                Y|
|R2RB1HML8N712P|          5|           26|         28|   N|                Y|
| ROXA8XP5EI2KK|          5|           48|         49|   N|                N|
|R2N92CUM7WBA1A|          5|           27|         28|   N|                Y|
|R2TAFZNG4KHFQB|          4|           24|         24|   N|     

In [10]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
total_reviews_count = vine_review_N_df.count() + vine_review_Y_df.count()
total_reviews_count

7793

In [14]:
# Determine the total number of 5-star reviews
five_star_reviews = more_helpful_vine_df.filter(more_helpful_vine_df["star_rating"] == "5")
five_star_reviews.count()

4041

In [15]:
# Determine the number of five star reviews when not in Vine program
five_star_reviews_vine_N = vine_review_N_df.filter(vine_review_N_df["star_rating"] == "5")
five_star_reviews_vine_N.count()

4027

In [16]:
# Determine the number of five star reviews when in Vine program
five_star_reviews_vine_Y = vine_review_Y_df.filter(vine_review_Y_df["star_rating"] == "5")
five_star_reviews_vine_Y.count()

14

In [25]:
# Print percentage of five star reviews within those enrolled in Vine program
percentage_vine_Y_five_star = five_star_reviews_vine_Y.count()/vine_review_Y_df.count()
percentage_vine_Y_five_star = "{:.2%}".format(percentage_vine_Y_five_star)
percentage_vine_Y_five_star

'32.56%'

In [26]:
# Print percentage of five star reviews within those not enrolled in Vine program
percentage_vine_N_five_star = five_star_reviews_vine_N.count()/vine_review_N_df.count()
percentage_vine_N_five_star = "{:.2%}".format(percentage_vine_N_five_star)
percentage_vine_N_five_star

'51.96%'

In [27]:
#Printing results for five star reviews when enrolled in Vine program
print(f"The total number of reviews that were made by people who paid for the Vine program was {vine_review_Y_df.count()}.")
print(f"Of those reviews, {five_star_reviews_vine_Y.count()} were 5 star ratings.")
print(f"The percentage of people who paid for the Vine program and gave a 5 star rating was {percentage_vine_Y_five_star}.")

The total number of reviews that were made by people who paid for the Vine program was 43.
Of those reviews, 14 were 5 star ratings.
The percentage of people who paid for the Vine program and gave a 5 star rating was 32.56%.


In [28]:
#Printing results for five star reviews when not enrolled in Vine program
print(f"The total number of reviews that were made by people who did not pay for the Vine program was {vine_review_N_df.count()}.")
print(f"Of those reviews, {five_star_reviews_vine_N.count()} were 5 star ratings.")
print(f"The percentage of people who did not pay for the Vine program and gave a 5 star rating was {percentage_vine_N_five_star}.")

The total number of reviews that were made by people who did not pay for the Vine program was 7750.
Of those reviews, 4027 were 5 star ratings.
The percentage of people who did not pay for the Vine program and gave a 5 star rating was 51.96%.
